In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
import pandas as pd
import rasterio.plot

import overpy
import geopandas as gpd
# import ee
from shapely.geometry import Polygon, box
import os
# import geemap
import json
import multiprocessing



In [2]:
# os.chdir('/Users/yer1k/Documents/Master/Duke/2023 Fall/Capstone/solar_wastewater/30_result/')
os.chdir('/Users/yer1k/Library/CloudStorage/GoogleDrive-iamtsubasaozora@gmail.com/My Drive/Capstone Project/Data/')

In [3]:
api = overpy.Overpass()

In [10]:

# Fetch all wastewater treatment plants within California's boundary (# changed to Texas)
query = f"""
    area[admin_level=4]["name"="Arizona"]->.searchArea; 
    (
    way["man_made"="wastewater_plant"](area.searchArea);
    );
    (._;>;);
    out body;
    """

result = api.query(query)

plants = {}

for way in result.ways:
    # Use the name of the plant or its ID if the name is not available
    plant_name = way.tags.get("name", f"Plant_{way.id}")

    # Extract nodes lat and lon without id
    nodes_coords = [(node.lon, node.lat) for node in way.nodes]

    # get rid of "Decimal"  in the coordinates
    nodes_coords = [tuple(map(float, i)) for i in nodes_coords]
    
    plants[plant_name] = nodes_coords

# plants_with_official_name = {key: value for key, value in plants.items() if not key.startswith("Plant_")}


geoms = [Polygon(plants[key]) for key in plants]
df = gpd.GeoDataFrame({'WWTP_name': list(plants.keys()), 'geometry': geoms}, crs="EPSG:4326")
df["centroid"] = df.to_crs('+proj=cea').centroid.to_crs(epsg=4326)



In [11]:
df

,WWTP_name,geometry,centroid
0,24th Street Water Treatment Plant,"POLYGON ((-112.03022 33.53292, -112.03434 33.5...",POINT (-112.03251 33.52970)
1,Southeast WRF,"POLYGON ((-111.69424 33.38022, -111.69588 33.3...",POINT (-111.69546 33.38153)
2,South Tempe Water Plant,"POLYGON ((-111.89480 33.36221, -111.89708 33.3...",POINT (-111.89713 33.35923)
3,Plant_29231740,"POLYGON ((-112.58222 33.35924, -112.58222 33.3...",POINT (-112.58287 33.35867)
4,Plant_30133555,"POLYGON ((-112.10849 33.42238, -112.11284 33.4...",POINT (-112.11020 33.42530)
...,...,...,...
551,Plant_1191529380,"POLYGON ((-113.23664 34.59078, -113.23660 34.5...",POINT (-113.23677 34.59093)
552,Plant_1191529381,"POLYGON ((-113.23712 34.59068, -113.23708 34.5...",POINT (-113.23725 34.59083)
553,Plant_1191529424,"POLYGON ((-113.23964 34.58821, -113.23961 34.5...",POINT (-113.23993 34.58838)
554,Plant_1192930076,"POLYGON ((-110.31199 31.65411, -110.31201 31.6...",POINT (-110.31384 31.65197)


In [8]:
!ls

Archive                         Status Track.gsheet
New_Images                      WWTP Lists
PPT From Christine              tif_bounding_box
Satellite Imagery_Automated RGB


In [12]:
# use loop to open the image in tif folder, and plot the bounding box on it
# read all the file names in the folder
for file in os.listdir('tif_bounding_box/Images of All 50 States/Arizona'):
    try:
        with rasterio.open(os.path.join('tif_bounding_box/Images of All 50 States/Arizona', file)) as src:
            # if file.startswith('download'):
                # continue
            # Read the image data
            img = src.read([1, 2, 3])  # Reads the first three bands for RGB
            # file name without extension
            name = os.path.splitext(file)[0]
            # find the corresponding bounding box
            df_name = df.loc[df['WWTP_name'] == name, :]
            # plot the bounding box
            fig, ax = plt.subplots(figsize=(8,8))
            # Plot the raster
            ax = rasterio.plot.show(img, ax=ax, transform=src.transform)
            # Plot the geometry on the ax axes instance
            df_name.boundary.plot(ax=ax, color='red', linewidth=1.5)
            plt.axis('off')
            # save the figure
            plt.savefig(os.path.join('tif_bounding_box/Images of All 50 States/Arizona', name + '.png'), bbox_inches='tight', pad_inches=0)
            plt.close()
    except:
        print(f"fail to plot {file}")